# 1. Combinar archivos 
#### Archivo de enrgia extraida con archivo de empresas

In [1]:
import pandas as pd
import glob
import re
import os
from pathlib import Path

# === CONFIGURACIÓN ===
CENTRALES_FILE = './data/empresas_generadoras.xlsx'
INPUT_PATTERN = './downloads/extracted_energia_c_iny_*.xlsx'
OUTPUT_PREFIX = './pre_data/energia_centrales_'

# === FUNCIONES AUXILIARES ===

def detectar_fila_encabezado(df):
    for i, row in df.iterrows():
        if any(str(cell).strip().upper() == "CENTRAL" for cell in row):
            return i
    return 0

def normalizar_nombre(nombre, nombres_centrales, alias):
    x = str(nombre).strip()
    if x in alias:
        return alias[x]
    
    # Limpieza más profunda para coincidencias flexibles
    x_clean = re.sub(r'\W+', '', x).upper()
    
    for k in nombres_centrales:
        k_clean = re.sub(r'\W+', '', k).upper()
        if k_clean == x_clean:
            return k
    
    # Casos especiales para Aguaí
    if "AGUAI" in x_clean or "AGUAÍ" in x_clean:
        if "AUTOPRODUCTOR" in x_clean:
            return "Aguai (Autoproductor)"
        return "Aguaí Energia"
    
    return x

# === MAPA DE ALIAS DE CENTRALES ===
alias = {
    "Kanata en Arocagua": "Kanata ARO",
    "Kanata en Valle Hermoso": "Kanata VHE",
    "Misicuni en Arocagua": "Misicuni ARO",
    "Misicuni en Valle Hermoso": "Misicuni VHE",
    "Yunchara": "Yunchara",
    "Aguaí Energía": "Aguaí Energia",
    "AGUAÍ ENERGÍA S.A.": "Aguaí Energia",
    "Santa Cruz (Aguaí)": "Santa Cruz (Aguaí)",
    "RÍO ELÉCTRICO S.A.": "RIO ELECTRICO S.A.",
    "CHACO ENERGÍAS S.A.": "CHACO ENERGIAS S.A.",
    "RIOELEC S.A.": "RIO ELECTRICO S.A.",
}

# === PROCESAMIENTO DE ARCHIVOS ===

def procesar_archivos():
    try:
        df_centrales = pd.read_excel(CENTRALES_FILE)
        df_centrales['CENTRAL'] = df_centrales['CENTRAL'].astype(str).str.strip()

        if not {'CENTRAL', 'GENERADOR', 'TECNOLOGIA'}.issubset(df_centrales.columns):
            print("Error: El archivo debe contener columnas 'CENTRAL', 'GENERADOR' y 'TECNOLOGIA'")
            return

        mapeo_generadores = dict(zip(df_centrales['CENTRAL'], df_centrales['GENERADOR']))
        mapeo_tecnologia = dict(zip(df_centrales['CENTRAL'], df_centrales['TECNOLOGIA']))
        nombres_centrales = set(df_centrales['CENTRAL'])

        # Asegurar centrales de Aguaí en el mapeo
        for central_aguai in ["Aguaí Energia", "Aguai (Autoproductor)"]:
            if central_aguai not in mapeo_generadores:
                mapeo_generadores[central_aguai] = "AGUAÍ ENERGÍA S.A."
                mapeo_tecnologia[central_aguai] = "Biomasa"

    except Exception as e:
        print(f"Error al cargar archivo de mapeo {CENTRALES_FILE}: {str(e)}")
        return

    for input_file in glob.glob(INPUT_PATTERN):
        file_number = re.search(r'extracted_energia_c_iny_(\d+)\.xlsx', input_file)
        if not file_number:
            continue

        file_number = file_number.group(1)
        output_file = f"{OUTPUT_PREFIX}{file_number}.xlsx"

        if os.path.exists(output_file):
            print(f"[Omitido] {output_file} ya existe.")
            continue

        try:
            df_raw = pd.read_excel(input_file, header=None)
            start_row = detectar_fila_encabezado(df_raw)
            df = pd.read_excel(input_file, skiprows=start_row)
            df.columns = [str(col).strip() for col in df.columns]

            central_col = next((c for c in df.columns if 'central' in c.lower() or 'agente' in c.lower()), None)
            if central_col and central_col != 'CENTRAL':
                df = df.rename(columns={central_col: 'CENTRAL'})
            if 'CENTRAL' not in df.columns:
                print(f"[Error] No se encontró columna 'CENTRAL' en {input_file}")
                continue

            df['CENTRAL'] = df['CENTRAL'].astype(str).str.strip()

            # Eliminación de filas basura (incluyendo totales de Aguaí)
            pattern_basura = r'TOTAL|TOTALES|Nota|Tipo de cambio|nan|CARGOS POR INYECCIONES|TOTAL\s*-\s*AGUAI'
            df = df[~df['CENTRAL'].str.contains(pattern_basura, case=False, na=True, regex=True)]
            df = df[~df['CENTRAL'].str.match(r'^\d{4}-\d{2}-\d{2}', na=False)]
            df = df[~df['CENTRAL'].str.upper().str.contains('CENTRAL\s*ENERGIA|POTENCIA', na=False)]
            df = df[df['CENTRAL'].notna() & (df['CENTRAL'] != '')]

            # Identificación de filas válidas
            df['CENTRAL_CLEAN'] = df['CENTRAL'].str.strip().str.upper()
            centrales_validas = set(x.upper() for x in nombres_centrales)
            first_valid_idx = df[df['CENTRAL_CLEAN'].isin(centrales_validas)].index.min()
            if pd.isna(first_valid_idx):
                print(f"[Error] No se encontraron centrales válidas en {input_file}")
                continue
            df = df.loc[first_valid_idx:].copy()

            # Normalización y mapeo
            df['CENTRAL_NORMALIZADA'] = df['CENTRAL'].apply(
                lambda x: normalizar_nombre(x, nombres_centrales, alias)
            )
            df['GENERADOR'] = df['CENTRAL_NORMALIZADA'].map(mapeo_generadores)
            df['TECNOLOGIA'] = df['CENTRAL_NORMALIZADA'].map(mapeo_tecnologia)

            # Forzar presencia de ambas centrales Aguaí
            aguai_centrales = ["Aguaí Energia", "Aguai (Autoproductor)"]
            for central in aguai_centrales:
                if central not in df['CENTRAL_NORMALIZADA'].values:
                    nueva_fila = {
                        'CENTRAL': central,
                        'CENTRAL_NORMALIZADA': central,
                        'GENERADOR': "AGUAÍ ENERGÍA S.A.",
                        'TECNOLOGIA': "Biomasa"
                    }
                    for col in df.columns:
                        if 'kW' in col or 'kWh' in col:
                            nueva_fila[col] = 0.0
                    df = pd.concat([df, pd.DataFrame([nueva_fila])], ignore_index=True)

            # Procesamiento numérico
            for col in df.columns:
                if 'kW' in col or 'kWh' in col:
                    df[col] = (
                        df[col]
                        .astype(str)
                        .str.replace(',', '')
                        .str.replace(' ', '')
                        .replace('nan', None)
                        .astype(float)
                    )

            # ... código existente ...

            columnas_finales = ['CENTRAL_NORMALIZADA', 'GENERADOR', 'TECNOLOGIA'] + [
                c for c in df.columns if c not in ['CENTRAL', 'CENTRAL_NORMALIZADA', 'CENTRAL_CLEAN', 'GENERADOR', 'TECNOLOGIA']
            ]
            df_final = df[columnas_finales].rename(columns={'CENTRAL_NORMALIZADA': 'CENTRAL'})

            # === NUEVO: RENOMBRAR COLUMNAS ESPECÍFICAS ===
            rename_columns = {
                'Energía': 'Energía kWh',
                'Potencia Firme Remunerada': 'Potencia kW'
            }
            df_final = df_final.rename(columns=rename_columns)

# ... resto del código existente ...
            # Guardar el DataFrame final
            df_final.to_excel(output_file, index=False)

            print(f"[OK] {input_file} → {output_file}")

            # Detectar centrales sin mapeo
            faltantes = df_final[df_final['GENERADOR'].isna()]['CENTRAL'].unique()
            if len(faltantes) > 0:
                print(f"  ⚠️ {len(faltantes)} centrales sin GENERADOR: {faltantes[:3]}{'...' if len(faltantes) > 3 else ''}")

        except Exception as e:
            print(f"[Error] {input_file}: {str(e)}")
            try:
                Path('./errors').mkdir(exist_ok=True)
                df.to_excel(f"./errors/ERROR_{file_number}.xlsx", index=False)
            except Exception as inner_e:
                print(f"Error al guardar archivo de error: {inner_e}")

# === EJECUCIÓN PRINCIPAL ===
if __name__ == "__main__":
    print("🔄 Iniciando procesamiento de archivos...")
    procesar_archivos()
    print("✅ Proceso completado.")

🔄 Iniciando procesamiento de archivos...
[OK] ./downloads\extracted_energia_c_iny_0123.xlsx → ./pre_data/energia_centrales_0123.xlsx
[OK] ./downloads\extracted_energia_c_iny_0124.xlsx → ./pre_data/energia_centrales_0124.xlsx
  ⚠️ 1 centrales sin GENERADOR: ['Yunchará']
[OK] ./downloads\extracted_energia_c_iny_0125.xlsx → ./pre_data/energia_centrales_0125.xlsx
[OK] ./downloads\extracted_energia_c_iny_0223.xlsx → ./pre_data/energia_centrales_0223.xlsx
[OK] ./downloads\extracted_energia_c_iny_0224.xlsx → ./pre_data/energia_centrales_0224.xlsx
  ⚠️ 1 centrales sin GENERADOR: ['Yunchará']
[OK] ./downloads\extracted_energia_c_iny_0225.xlsx → ./pre_data/energia_centrales_0225.xlsx
[OK] ./downloads\extracted_energia_c_iny_0323.xlsx → ./pre_data/energia_centrales_0323.xlsx
  ⚠️ 5 centrales sin GENERADOR: ['San Borja' 'Rurrenabaque' 'Yucumo']...
[OK] ./downloads\extracted_energia_c_iny_0324.xlsx → ./pre_data/energia_centrales_0324.xlsx
[OK] ./downloads\extracted_energia_c_iny_0325.xlsx → ./pre_d

# 2. Merge archivos energia

In [2]:
import pandas as pd
import glob
from datetime import datetime

def consolidar_en_formato_largo():
    archivos = sorted(glob.glob('./pre_data/energia_centrales_*.xlsx'))
    if not archivos:
        print("No se encontraron archivos.")
        return

    registros = []

    for archivo in archivos:
        try:
            # Extraer mes y año del nombre del archivo
            periodo = archivo.split('_')[-1].split('.')[0]
            mes = int(periodo[:2])
            año = 2000 + int(periodo[2:])
            fecha = datetime(año, mes, 1)

            df = pd.read_excel(archivo)

            # Verificar columnas esenciales
            if 'CENTRAL' not in df.columns:
                print(f"Omitido: {archivo} no tiene columna CENTRAL.")
                continue

            columnas_necesarias = ['CENTRAL', 'GENERADOR']
            columnas_datos = [col for col in df.columns if col not in columnas_necesarias + ['TECNOLOGIA']]
            
            # Si no hay columnas de datos, omitir
            if not columnas_datos:
                print(f"Omitido: {archivo} no tiene columnas de datos.")
                continue

            # Si no existe columna TECNOLOGIA, crear una con NaN
            if 'TECNOLOGIA' not in df.columns:
                df['TECNOLOGIA'] = None

            # Derretir (melt) el DataFrame para convertir a formato largo
            temp = pd.melt(
                df,
                id_vars=['CENTRAL', 'GENERADOR', 'TECNOLOGIA'],
                value_vars=columnas_datos,
                var_name='VARIABLE',
                value_name='VALOR'
            )
            
            temp['FECHA'] = fecha
            registros.append(temp)

        except Exception as e:
            print(f"Error procesando {archivo}: {e}")
            continue

    if not registros:
        print("No se pudo consolidar ningún archivo válido.")
        return

    df_largo = pd.concat(registros, ignore_index=True)
    df_largo = df_largo[['FECHA', 'CENTRAL', 'GENERADOR', 'TECNOLOGIA', 'VARIABLE', 'VALOR']]

    df_largo.to_excel("./preprocess/serie_temporal_larga.xlsx", index=False)
    print("Consolidación completada en formato largo.")
    print(f"Filas totales: {len(df_largo)}")
    print("Archivo guardado como 'serie_temporal_larga.xlsx'")

if __name__ == "__main__":
    consolidar_en_formato_largo()


Consolidación completada en formato largo.
Filas totales: 2534
Archivo guardado como 'serie_temporal_larga.xlsx'


# 3. Creacion de serie de tiempo de energia

In [3]:
import pandas as pd

# Cargar archivo en formato largo
df = pd.read_excel("./preprocess/serie_temporal_larga.xlsx")

# Asegurar que FECHA sea tipo datetime
df['FECHA'] = pd.to_datetime(df['FECHA'])

# Formatear fecha como MMYYYY (ejemplo: 012023)
df['PERIODO'] = df['FECHA'].dt.strftime("%m%Y")

# Crear columna combinada: VARIABLE + PERIODO
df['COLUMNA'] = df['VARIABLE'] + ' ' + df['PERIODO']

# Pivotear: una fila por CENTRAL, GENERADOR, TECNOLOGÍA
tabla_pivot = df.pivot_table(
    index=['CENTRAL', 'GENERADOR', 'TECNOLOGIA'],
    columns='COLUMNA',
    values='VALOR',
    aggfunc='sum'  # En caso de duplicados, los suma
)

# Resetear índice para que sea un DataFrame plano
tabla_pivot = tabla_pivot.reset_index()

# --- NUEVO CÓDIGO PARA ORDENAR COLUMNAS CRONOLÓGICAMENTE ---
# 1. Separar las columnas en componentes
fixed_cols = ['CENTRAL', 'GENERADOR', 'TECNOLOGIA']
other_cols = [c for c in tabla_pivot.columns if c not in fixed_cols]

# 2. Crear DataFrame temporal para ordenación
cols_df = pd.DataFrame({
    'col_name': other_cols,
    'variable': [c.split()[0] for c in other_cols],  # Ej: 'Energía MWh'
    'periodo': [c.split()[-1] for c in other_cols]   # Ej: '012023'
})

# 3. Convertir periodo a fecha para ordenar correctamente
cols_df['fecha'] = pd.to_datetime(cols_df['periodo'], format='%m%Y')

# 4. Ordenar por: 1. Tipo de variable, 2. Fecha cronológica
cols_df = cols_df.sort_values(by=['variable', 'fecha'])

# 5. Construir lista final de columnas ordenadas
ordered_columns = fixed_cols + cols_df['col_name'].tolist()

# 6. Aplicar nuevo orden al DataFrame
tabla_pivot = tabla_pivot[ordered_columns]
# --- FIN DEL NUEVO CÓDIGO ---

# Guardar resultado
tabla_pivot.to_excel("./preprocess/serie_temporal_pivotada.xlsx", index=False)
print("✅ Archivo guardado como 'serie_temporal_pivotada.xlsx'")





✅ Archivo guardado como 'serie_temporal_pivotada.xlsx'


# 4. Depuracion de serie de tiempo de energia

In [4]:
import pandas as pd

# Cargar archivo
df = pd.read_excel("./preprocess/serie_temporal_pivotada.xlsx")

# 1. Definir columnas base (fijas)
fixed_cols = ['CENTRAL', 'GENERADOR', 'TECNOLOGIA']

# 2. Filtrar y ordenar columnas de energía
energy_cols = [col for col in df.columns if col.startswith('Energía kWh')]

# 3. Ordenar cronológicamente por fecha
energy_cols_sorted = sorted(
    energy_cols,
    key=lambda x: (pd.to_datetime(x.split()[-1], format='%m%Y'))
)

# 4. Crear nuevo DataFrame con columnas seleccionadas
df_energy = df[fixed_cols + energy_cols_sorted]

df_energy.head()

# Guardar resultado
df_energy.to_excel("./data/serie_energia.xlsx", index=False)
print("✅ Archivo guardado como 'serie_energia.xlsx'")


✅ Archivo guardado como 'serie_energia.xlsx'


# 5. Depuracion de serie de tiempo de potencia

In [5]:
import pandas as pd

# Cargar archivo
df = pd.read_excel("./preprocess/serie_temporal_pivotada.xlsx")

# 1. Definir columnas base (fijas)
fixed_cols = ['CENTRAL', 'GENERADOR', 'TECNOLOGIA']

# 2. Filtrar y ordenar columnas de energía
energy_cols = [col for col in df.columns if col.startswith('Potencia kW')]

# 3. Ordenar cronológicamente por fecha
energy_cols_sorted = sorted(
    energy_cols,
    key=lambda x: (pd.to_datetime(x.split()[-1], format='%m%Y'))
)

# 4. Crear nuevo DataFrame con columnas seleccionadas
df_energy = df[fixed_cols + energy_cols_sorted]

df_energy.head()

# Guardar resultado
df_energy.to_excel("./data/serie_potencia.xlsx", index=False)
print("✅ Archivo guardado como 'serie_potencia.xlsx'")



✅ Archivo guardado como 'serie_potencia.xlsx'
